# Data merging of FFIEC and UBFR Reports

### Purpose: To get a clean, workable, datafile. 

#### By: Jesus Plascencia 
#### On: Oct 8, 2025

In [109]:
!pip install missingno
pip install pyjanitor

In [3]:
import pandas as pd
import os
import missingno as msno
import janitor

# Loading Banking data, going from pickle to Parquet
Reading this dataframe is very time consuming, I'm going to use parquet, I might even load in specific snippets just for my mac. It took minutes to load using pickle, now it takes 51 seconds so its more efficient.

Once I fully understand each column I'll load in specific columns. For now, I really want to understand what it is im looking at


In [5]:
#Ask bieri how to make a global path 
#df = pd.read_pickle(r'/Users/jesusplascencia/Desktop/Indepndent_data/bank_data_2002_to_2023.pkl')
#df.to_parquet(r'/Users/jesusplascencia/Desktop/Indepndent_data/bank_data_2002_to_2023.parquet')
#var_def = pd.read_csv(r"C:\Users\jesus\Desktop\VariableDefinitions.csv")

df = pd.read_parquet(r'/Users/jesusplascencia/Desktop/Indepndent_data/bank_data_2002_to_2023.parquet')



In [22]:
df.head(2)

,Reporting Period End Date,IDRSSD,FDIC Certificate Number,OCC Charter Number,OTS Docket Number,Primary ABA Routing Number,Financial Institution Name,Financial Institution Address,Financial Institution City,Financial Institution State,...,RIADHT69,RIADHT70,RIADHT73,RIADHT74,RCFD2143,RCFDJA22,RCON2143,RCONJJ34,RIADJJ33,RCFDJJ34
0,2002-12-31,37.0,10057.0,0.0,16553.0,61107146.0,BANK OF HANCOCK COUNTY,321 BROAD STREET,SPARTA,GA,...,None,None,None,None,None,None,None,None,None,None
1,2002-12-31,242.0,3850.0,0.0,12627.0,81220537.0,FIRST COMMUNITY BANK XENIA-FLORA,FRONT STREET,XENIA,IL,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
df.describe()

In [ ]:
df.dtypes

In [115]:
col_list= list(df.columns)
print(col_list)

['Reporting Period End Date', 'IDRSSD', 'FDIC Certificate Number', 'OCC Charter Number', 'OTS Docket Number', 'Primary ABA Routing Number', 'Financial Institution Name', 'Financial Institution Address', 'Financial Institution City', 'Financial Institution State', 'Financial Institution Zip Code', 'Financial Institution Filing Type', 'Last Date/Time Submission Updated On', 'RIAD4180', 'RIAD4185', 'RIAD4200', 'RIAD4217', 'RIAD4230', 'RIAD4300', 'RIAD4301', 'RIAD4302', 'RIAD4313', 'RIAD4320', 'RIAD4340', 'RIAD4507', 'RIAD4508', 'RIAD4513', 'RIAD4518', 'RIAD5415', 'RIAD5416', 'RIAD8431', 'RIAD8757', 'RIAD8758', 'RIAD8759', 'RIAD8760', 'RIAD8761', 'RIAD8762', 'RIAD8763', 'RIAD9106', 'RIADA220', 'RIADA251', 'RIADA517', 'RIADA518', 'RIADA530', 'RIADB485', 'RIADB486', 'RIADB487', 'RIADB488', 'RIADB489', 'RIADB490', 'RIADB491', 'RIADB492', 'RIADB493', 'RIADB494', 'RIADB496', 'RIADB497', 'RIADC216', 'RIADC232', 'RCFD0071', 'RCFD0081', 'RCFD0426', 'RCFD1248', 'RCFD1249', 'RCFD1250', 'RCFD1251', '

In [51]:
reviewing_na = df.isna().mean().sort_values(ascending=False)
reviewing_na.head(10)



RIADFT36    0.998645
RIADFT37    0.998645
RIADFT38    0.998643
RIADFT40    0.998643
RIADFT39    0.998643
RIADK094    0.998609
RIADK090    0.998609
RCFDJJ34    0.997241
RCFDJA22    0.996938
RCFD2143    0.996858
dtype: float64

#### Variables have nots of NaN, understanding the data w/defintions

Date column is currently an object I'm just making sure the dataset is in the correct format for me to do analysis 

In [49]:
print(df.isna().sum())

print(df['Last Date/Time Submission Updated On'].dtypes)

Reporting Period End Date         0
IDRSSD                            0
FDIC Certificate Number           0
OCC Charter Number                0
OTS Docket Number                 0
                              ...  
RCFDJA22                     589304
RCON2143                     476864
RCONJJ34                     493214
RIADJJ33                     491583
RCFDJJ34                     589483
Length: 630, dtype: int64
object


In [51]:
df['Last Date/Time Submission Updated On'].head(10)

0    2005-08-08T16:16:33
1    2005-08-08T16:16:33
2    2005-08-08T16:16:33
3    2005-08-08T16:16:33
4    2005-08-08T16:16:33
5    2005-08-08T16:16:33
6    2005-08-08T16:16:33
7    2005-08-08T16:16:33
8    2005-08-08T16:16:33
9    2005-08-08T16:16:33
Name: Last Date/Time Submission Updated On, dtype: object

#### Converting from object to a timestamp

In [11]:
df['date'] = pd.to_datetime(df['Last Date/Time Submission Updated On'], yearfirst=True)
df['date'].dtype

dtype('<M8[ns]')

In [13]:
df['date'].head(1)

0   2005-08-08 16:16:33
Name: date, dtype: datetime64[ns]

**Range of the data**

In [16]:
df.date.min(),df.date.max()

(Timestamp('2005-08-05 02:06:21'), Timestamp('2024-10-11 16:17:56'))

In [18]:
print(f"""
Date: {df.date.iloc[0]}
Day of year: {df.date.dt.day_of_year.iloc[0]}
Day of week: {df.date.dt.dayofweek.iloc[0]}
Month: {df.date.dt.month.iloc[0]}
Month Name: {df.date.dt.month_name().iloc[0]}
Quarter: {df.date.dt.quarter.iloc[0]}
Year: {df.date.dt.year.iloc[0]}
ISO Week: {df.date.dt.isocalendar().week.iloc[0]}
""")


Date: 2005-08-08 16:16:33
Day of year: 220
Day of week: 0
Month: 8
Month Name: August
Quarter: 3
Year: 2005
ISO Week: 32



**missing values?**

In [46]:
df['RIADHT70'] = pd.to_numeric(df['RIADHT70'], errors='coerce')

df['RIADHT70'].dtype

dtype('float64')

In [42]:

plot_df = pd.pivot_table(df, index="date", columns="IDRSSD", values="RIADHT70")
# Generate Plot. Since we have a datetime index, we can mention the frequency to decide what do we want on the X axis
msno.matrix(plot_df, freq="M", fontsize=20)
#plt.savefig('imgs/chapter_2/missing_no_full.png', bbox_inches='tight')
plt.show()

KeyError: 'date'

# Loading Bank Failures --> Creating Dummy Variables and mergiing

In [6]:
df_failures = pd.read_csv(r'/Users/jesusplascencia/Desktop/Indepndent_data/Banking Failures.csv')

In [9]:
df_failures.head(10) 

,Bank Name,City,State,Cert,Acquiring Institution,Closing Date,Fund
0,Pulaski Savings Bank,Chicago,IL,28611,Millennium Bank,17-Jan-25,10548
1,First National Bank of Lindsay,Lindsay,OK,4134,First Bank & Trust Co.,18-Oct-24,10547
2,Republic First Bank dba Republic Bank,Philadelphia,PA,27332,"Fulton Bank, National Association",26-Apr-24,10546
3,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,3-Nov-23,10545
4,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.",28-Jul-23,10544
5,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.",1-May-23,10543
6,Signature Bank,New York,NY,57053,"Flagstar Bank, N.A.",12-Mar-23,10540
7,Silicon Valley Bank,Santa Clara,CA,24735,First–Citizens Bank & Trust Company,10-Mar-23,10539
8,Almena State Bank,Almena,KS,15426,Equity Bank,23-Oct-20,10538
9,First City Bank of Florida,Fort Walton Beach,FL,16748,"United Fidelity Bank, fsb",16-Oct-20,10537


In [56]:
df['Financial Institution Name'].unique()

array(['BANK OF HANCOCK COUNTY', 'FIRST COMMUNITY BANK XENIA-FLORA',
       'MINEOLA COMMUNITY BANK, SSB', ...,
       'TIAA TRUST, NATIONAL ASSOCIATION', 'COMMUNITY UNITY BANK',
       'CONNECTIONS BANK MIDWEST'], dtype=object)

In [58]:
df['Financial Institution City'].unique()

array(['SPARTA', 'XENIA', 'MINEOLA', ..., 'FLOWOOD', 'BEE CAVE', 'GAS'],
      dtype=object)

In [88]:
#print(df_failures['city_'].unique())

In [40]:
df_failures.isna().sum()   

Bank Name                 0
City                      0
State                     0
Cert                      0
Acquiring Institution     0
Closing Date              0
Fund                      0
dtype: int64

In [86]:
df_failures.dtypes

bank_name_                object
city_                     object
state_                    object
cert_                      int64
acquiring_institution_    object
closing_date_             object
fund                       int64
dtype: object

In [84]:
print(df_failures.columns)

Index(['bank_name_', 'city_', 'state_', 'cert_', 'acquiring_institution_',
       'closing_date_', 'fund'],
      dtype='object')


In [52]:
print(df.columns)

Index(['Reporting Period End Date', 'IDRSSD', 'FDIC Certificate Number',
       'OCC Charter Number', 'OTS Docket Number', 'Primary ABA Routing Number',
       'Financial Institution Name', 'Financial Institution Address',
       'Financial Institution City', 'Financial Institution State',
       ...
       'RIADHT69', 'RIADHT70', 'RIADHT73', 'RIADHT74', 'RCFD2143', 'RCFDJA22',
       'RCON2143', 'RCONJJ34', 'RIADJJ33', 'RCFDJJ34'],
      dtype='object', length=630)


### I got tired of cleaning the names so I cheated and used Janitor function

In [19]:
df_failures = df_failures.clean_names()

In [21]:
list(df_failures.columns) 

['bank_name_',
 'city_',
 'state_',
 'cert_',
 'acquiring_institution_',
 'closing_date_',
 'fund']

### Cert and FDIC Certificate Number May be the best way to merge the datasets

It definetely is the best way based off the overlapping Identifyers

In [24]:
cert_uniq = list(df_failures['cert_'].unique())
certifiace_number_uniq = list(df_failures['cert_'].unique())

In [26]:
len(certifiace_number_uniq)
len(cert_uniq)



571

In [28]:
overlapping_certs = []
for i in cert_uniq:
    if i in certifiace_number_uniq:
        overlapping_certs.append(i)

len(overlapping_certs)

571

### Left merge to keep failing banks and those that arent failing, I also will make a dummy variable and based off that I'll begin analyzing "variable definitions" thorughly to identify a sound way to approach our main objective

In [31]:
df_failed_banks = pd.merge(df, df_failures, left_on='FDIC Certificate Number', right_on='cert_')

In [32]:
df_failed_banks.shape

(16623, 637)

In [33]:
df_merged =  pd.merge(df, df_failures, left_on='FDIC Certificate Number', right_on='cert_', how = 'left')

In [36]:
df_merged

,Reporting Period End Date,IDRSSD,FDIC Certificate Number,OCC Charter Number,OTS Docket Number,Primary ABA Routing Number,Financial Institution Name,Financial Institution Address,Financial Institution City,Financial Institution State,...,RCONJJ34,RIADJJ33,RCFDJJ34,bank_name_,city_,state_,cert_,acquiring_institution_,closing_date_,fund
0,2002-12-31,37.0,10057.0,0.0,16553.0,61107146.0,BANK OF HANCOCK COUNTY,321 BROAD STREET,SPARTA,GA,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002-12-31,242.0,3850.0,0.0,12627.0,81220537.0,FIRST COMMUNITY BANK XENIA-FLORA,FRONT STREET,XENIA,IL,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002-12-31,279.0,28868.0,0.0,2523.0,311972526.0,"MINEOLA COMMUNITY BANK, SSB",215 W BROAD,MINEOLA,TX,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002-12-31,354.0,14083.0,0.0,0.0,101107475.0,BISON STATE BANK,MAIN AND WALNUT,BISON,KS,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002-12-31,439.0,16498.0,0.0,0.0,61212387.0,PEOPLES BANK,MAIN STREET,BLACKSHEAR,GA,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591109,2023-03-31,5784921.0,59312.0,0.0,0.0,21315169.0,GS&L MUNICIPAL BANK,42 CHURCH STREET,GOUVERNEUR,NY,...,0,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
591110,2023-03-31,5787418.0,59289.0,0.0,0.0,11402121.0,WALDEN MUTUAL BANK,66 N MAIN STREET,CONCORD,NH,...,5855,76,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
591111,2023-03-31,5788705.0,59296.0,0.0,0.0,122245387.0,BANK IRVINE,"6201 OAK CANYON, SUITE 150",IRVINE,CA,...,0,212,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
591112,2023-03-31,5805451.0,57288.0,25281.0,0.0,122106497.0,"WESTERN ALLIANCE TRUST COMPANY, NATIONAL ASSOC...",1 EAST WASHINGTON STREET SUITE 1400,PHOENIX,AZ,...,0,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
failed_certs_dummy_nonunique = df_failures['cert_'].dropna().astype(int)
failed_certs_dummy_unique = failed_certs_dummy_nonunique.unique()

df['failed_banks'] = df['FDIC Certificate Number'].isin(failed_certs_dummy_unique).astype(int)


In [38]:
df_merged['failed_banks'] = df_merged['FDIC Certificate Number'].isin(failed_certs_dummy_unique).astype(int)


In [39]:
print(df.shape)
print(df_failures.shape)
print(df_merged.shape)


(591114, 631)
(571, 7)
(591114, 638)


In [41]:
len(df_merged[df_merged['failed_banks'] == 1])
df_failed_banks.shape


(16623, 637)

In [47]:
print(list(df.columns))

['Reporting Period End Date', 'IDRSSD', 'FDIC Certificate Number', 'OCC Charter Number', 'OTS Docket Number', 'Primary ABA Routing Number', 'Financial Institution Name', 'Financial Institution Address', 'Financial Institution City', 'Financial Institution State', 'Financial Institution Zip Code', 'Financial Institution Filing Type', 'Last Date/Time Submission Updated On', 'RIAD4180', 'RIAD4185', 'RIAD4200', 'RIAD4217', 'RIAD4230', 'RIAD4300', 'RIAD4301', 'RIAD4302', 'RIAD4313', 'RIAD4320', 'RIAD4340', 'RIAD4507', 'RIAD4508', 'RIAD4513', 'RIAD4518', 'RIAD5415', 'RIAD5416', 'RIAD8431', 'RIAD8757', 'RIAD8758', 'RIAD8759', 'RIAD8760', 'RIAD8761', 'RIAD8762', 'RIAD8763', 'RIAD9106', 'RIADA220', 'RIADA251', 'RIADA517', 'RIADA518', 'RIADA530', 'RIADB485', 'RIADB486', 'RIADB487', 'RIADB488', 'RIADB489', 'RIADB490', 'RIADB491', 'RIADB492', 'RIADB493', 'RIADB494', 'RIADB496', 'RIADB497', 'RIADC216', 'RIADC232', 'RCFD0071', 'RCFD0081', 'RCFD0426', 'RCFD1248', 'RCFD1249', 'RCFD1250', 'RCFD1251', '

In [49]:
df_merged.to_parquet(r'/Users/jesusplascencia/Desktop/Indepndent_data/bank_data_failed.parquet')


## Now adding the banks that went inactive for any reason

In [51]:
df_inactive = pd.read_csv(r'/Users/jesusplascencia/Desktop/Indepndent_data/BanksEndBeforeDate.csv')


In [52]:
df_inactive.head(10)

,name,city,state,cert,id_number,end_date,Unnamed: 6
0,PEOPLES BANK,BLACKSHEAR,GA,16498,439,2007-06-30 00:00:00,NaN
1,PULASKI BANK AND TRUST COMPANY,LITTLE ROCK,AR,17562,1146,2007-03-31 00:00:00,NaN
2,FIRST STATE BANK KIOWA KANSAS THE,KIOWA,KS,15328,1557,2017-12-31 00:00:00,NaN
3,SEA ISLAND BANK,STATESBORO,GA,5707,1632,2010-03-31 00:00:00,NaN
4,ALLEGHENY VALLEY BANK OF PITTSBURGH,PITTSBURGH,PA,6058,1829,2017-03-31 00:00:00,NaN
5,FARMERS & MERCHANTS BANK,TOMAH,WI,13046,2040,2019-06-30 00:00:00,NaN
6,ORITANI SAVINGS BANK,HACKENSACK,NJ,28866,2376,2019-09-30 00:00:00,NaN
7,BEVERLY STATE BANK,BEVERLY,KS,10506,2451,2010-06-30 00:00:00,NaN
8,STATE BANK OF MARIETTA,MARIETTA,MN,18525,2554,2019-03-31 00:00:00,NaN
9,SECURITY BANK AND TRUST COMPANY OF ALBANY,ALBANY,GA,20246,2938,2010-03-31 00:00:00,NaN


In [53]:
df_inactive.dtypes

name           object
city           object
state          object
cert            int64
id_number       int64
end_date       object
Unnamed: 6    float64
dtype: object

In [54]:
df_inactive.info

<bound method DataFrame.info of                                           name         city state   cert  \
0                                 PEOPLES BANK   BLACKSHEAR    GA  16498   
1               PULASKI BANK AND TRUST COMPANY  LITTLE ROCK    AR  17562   
2         FIRST STATE BANK  KIOWA  KANSAS  THE        KIOWA    KS  15328   
3                              SEA ISLAND BANK   STATESBORO    GA   5707   
4          ALLEGHENY VALLEY BANK OF PITTSBURGH   PITTSBURGH    PA   6058   
...                                        ...          ...   ...    ...   
5803                       NEW TRADITIONS BANK      ORLANDO    FL  58822   
5804            INVESCO NATIONAL TRUST COMPANY      ATLANTA    GA  59078   
5805                       BANK OF AUSTIN  THE       AUSTIN    TX  59104   
5806                            BLUE GATE BANK   COSTA MESA    CA  59094   
5807  PEAK TRUST COMPANY  NATIONAL ASSOCIATION    ANCHORAGE    AK  59322   

      id_number             end_date  Unnamed: 6  
0   

#### Cert has no NaN 

In [56]:
df_inactive.isna().sum()

name             0
city             0
state            0
cert             0
id_number        0
end_date         0
Unnamed: 6    5808
dtype: int64

In [57]:
df_inactive.columns

Index(['name', 'city', 'state', 'cert', 'id_number', 'end_date', 'Unnamed: 6'], dtype='object')

In [58]:
df.describe()

,Reporting Period End Date,IDRSSD,FDIC Certificate Number,OCC Charter Number,OTS Docket Number,Primary ABA Routing Number,Financial Institution Zip Code,Financial Institution Filing Type,failed_banks
count,591114,5.911140e+05,591114.000000,591114.000000,591114.000000,5.911140e+05,5.911140e+05,591114.000000,591114.000000
mean,2011-12-20 10:04:41.147799040,1.012928e+06,21049.988929,24016.094428,4402.678644,9.853230e+07,3.548824e+07,42.572032,0.028121
min,2002-03-31 00:00:00,3.700000e+01,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,31.000000,0.000000
25%,2006-06-30 00:00:00,3.287770e+05,8852.000000,0.000000,0.000000,6.700044e+07,3.764300e+04,41.000000,0.000000
50%,2011-06-30 00:00:00,6.589240e+05,16472.000000,0.000000,0.000000,8.430301e+07,5.710400e+04,41.000000,0.000000
75%,2016-12-31 00:00:00,9.786590e+05,29430.000000,0.000000,10686.000000,1.110245e+08,7.205800e+04,41.000000,0.000000
max,2023-12-31 00:00:00,5.860740e+06,91385.000000,718184.000000,80861.000000,3.252713e+08,9.936204e+08,51.000000,1.000000
std,NaN,1.035594e+06,17122.238628,120011.383852,6603.031598,5.854406e+07,1.526362e+08,4.026490,0.165320


In [76]:
 df_bank_failure_inactive= pd.merge(df_merged, df_inactive, left_on='FDIC Certificate Number', right_on='cert', how = 'left')

In [60]:
inactive_certs_dummy_nonunique = df_inactive['cert'].dropna().astype(int)
inactive_certs_dummy_unique = inactive_certs_dummy_nonunique.unique()

df['inactive_banks'] = df['FDIC Certificate Number'].isin(inactive_certs_dummy_unique).astype(int)
df_merged['inactive_banks'] = df_merged['FDIC Certificate Number'].isin(inactive_certs_dummy_unique).astype(int)



In [330]:
df_merged.to_parquet(r'/Users/jesusplascencia/Desktop/Indepndent_data/bank_data_failed_inactive.parquet')